<a href="https://colab.research.google.com/github/RevanthReddy25/AI_Commentry_ClashRoyale/blob/main/AI_Commentry_ClashRoyale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# -----------------------------------
# ✅ Install Required Libraries
# -----------------------------------
!pip install -q transformers datasets sentencepiece gtts

# -----------------------------------
# ✅ Imports
# -----------------------------------
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import torch
import os
from gtts import gTTS
import IPython.display as ipd

# Disable wandb
os.environ["WANDB_DISABLED"] = "true"

# -----------------------------------
# ✅ Load Dataset (Upload First)
# -----------------------------------
# from google.colab import files
# files.upload()  # Upload commentary_data.csv

df = pd.read_csv("commentary_data.csv").dropna()
df = df.rename(columns={"input": "input_text", "target": "target_text"})
dataset = Dataset.from_pandas(df)

# -----------------------------------
# ✅ Tokenize Dataset
# -----------------------------------
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

def preprocess(example):
    input_ = tokenizer("Commentate: " + example["input_text"], truncation=True, padding="max_length", max_length=64)
    target = tokenizer(example["target_text"], truncation=True, padding="max_length", max_length=64)
    input_["labels"] = target["input_ids"]
    return input_

tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)

# -----------------------------------
# ✅ Load Model
# -----------------------------------
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# -----------------------------------
# ✅ Training Arguments
# -----------------------------------
training_args = TrainingArguments(
    output_dir="./t5_commentator",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch"
)

# -----------------------------------
# ✅ Trainer
# -----------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

# -----------------------------------
# ✅ Train the Model
# -----------------------------------
trainer.train()

# -----------------------------------
# ✅ Save the Model
# -----------------------------------
model.save_pretrained("t5_commentator")
tokenizer.save_pretrained("t5_commentator")

# -----------------------------------
# ✅ Inference + Voice
# -----------------------------------
def generate_commentary(text):
    input_ids = tokenizer("Commentate: " + text, return_tensors="pt").input_ids.to(model.device)
    output_ids = model.generate(input_ids, max_length=50)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

def speak(text):
    tts = gTTS(text)
    tts.save("commentary.mp3")
    return ipd.Audio("commentary.mp3")

# -----------------------------------
# ✅ Test Cases
# -----------------------------------
test_inputs = [
    "Player 2 drops Mini P.E.K.K.A on the right lane.",
    "Player 2 casts Fireball on Minions.",
    "Player 1 uses Log to clear Skeleton Army.",
    "Player 2 places P.E.K.K.A at the back.",
    "Player 1 freezes tower and troops!",
    "Player 2 destroys the left Princess Tower.",
    "Player 1 wins the game!",
]

for test in test_inputs:
    print(f"🎮 Input: {test}")
    commentary = generate_commentary(test)
    print(f"🗣️ Commentary: {commentary}\n")
    display(speak(commentary))


Map:   0%|          | 0/110 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
10,4.599200
20,1.665300
30,1.296000
40,1.111900
50,0.979100
60,0.864700
70,0.802300
80,0.740500
90,0.709800
100,0.688800


🎮 Input: Player 2 drops Mini P.E.K.K.A on the right lane.
🗣️ Commentary: Mini P.E.E.K.A reaches the right — a slick wing!



🎮 Input: Player 2 casts Fireball on Minions.
🗣️ Commentary: Fireball is a fireball — Player 2 catches fireball!



🎮 Input: Player 1 uses Log to clear Skeleton Army.
🗣️ Commentary: skeptics a squat! Log clears the Skeleton Army — a powerful move!



🎮 Input: Player 2 places P.E.K.K.A at the back.
🗣️ Commentary: P.E.K.A slaps the back — a slick slap — a big push from Player 2!



🎮 Input: Player 1 freezes tower and troops!
🗣️ Commentary: Player 1 freezes tower and troops!



🎮 Input: Player 2 destroys the left Princess Tower.
🗣️ Commentary: Princess Tower is a left-left stricken player 2 — Player 2's left wing — a powerful battle — a quick and powerful attack.



🎮 Input: Player 1 wins the game!
🗣️ Commentary: — Player 1 wins the game!

